In [19]:
#Correlation test vanilla on both real and synthetic data.
import scipy
import pandas as pd
import pickle
import numpy as np
from analysis_util import *


In [26]:
#Do VANILLA regression test here (across all models).
#DO I NEED TO REMOVE RANDOM SAMPLING HERE?
import statsmodels.api as sm
filehandler = open('parsed_results/synth_results.obj', 'rb') 
synth_results = pd.DataFrame.from_dict(pickle.load(filehandler))
aggregated_synth = aggregate_synthetic_results(synth_results)
selection = aggregated_synth[aggregated_synth['acquisition'] != "RS"]
xy = selection[['calibration_mse', 'tot_regret_pool']]
xy['intercept'] = 1
model = sm.OLS(xy['tot_regret_pool'], xy[['calibration_mse', 'intercept']])
results=model.fit()
print(results.summary())
#scipy.stats.linregress(aggregated_synth['calibration_mse'], aggregated_synth['inst_regret_pool'])

                            OLS Regression Results                            
Dep. Variable:        tot_regret_pool   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     824.1
Date:                Tue, 14 Feb 2023   Prob (F-statistic):          7.13e-179
Time:                        17:50:47   Log-Likelihood:            -1.7572e+05
No. Observations:               30720   AIC:                         3.514e+05
Df Residuals:                   30718   BIC:                         3.515e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
calibration_mse   266.8359      9.295     

/var/folders/pg/8kgknkg97zgbj393j1nl2gdh0000gn/T/ipykernel_33511/470351651.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xy['intercept'] = 1


In [27]:
#Regression test with additional intercept for each of the models.
import statsmodels.api as sm
aggregated_synth = aggregate_synthetic_results(synth_results)
selection = aggregated_synth[aggregated_synth['acquisition'] != "RS"]
xy = selection[['calibration_mse', 'tot_regret_pool']]
xy[['BNN', 'DE', 'GP', 'RF']] = pd.get_dummies(selection['surrogate'])[['BNN', 'DE', 'GP', 'RF']]
model = sm.OLS(xy['tot_regret_pool'], xy[['calibration_mse', 'BNN', 'DE', 'GP', 'RF']])
results=model.fit()
print(results.summary())
#scipy.stats.linregress(aggregated_synth['calibration_mse'], aggregated_synth['inst_regret_pool'])

                            OLS Regression Results                            
Dep. Variable:        tot_regret_pool   R-squared:                       0.108
Model:                            OLS   Adj. R-squared:                  0.108
Method:                 Least Squares   F-statistic:                     930.2
Date:                Tue, 14 Feb 2023   Prob (F-statistic):               0.00
Time:                        17:50:50   Log-Likelihood:            -1.7437e+05
No. Observations:               30720   AIC:                         3.487e+05
Df Residuals:                   30715   BIC:                         3.488e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
calibration_mse  -251.9339     13.767    -

/var/folders/pg/8kgknkg97zgbj393j1nl2gdh0000gn/T/ipykernel_33511/2587967697.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xy[['BNN', 'DE', 'GP', 'RF']] = pd.get_dummies(selection['surrogate'])[['BNN', 'DE', 'GP', 'RF']]
/var/folders/pg/8kgknkg97zgbj393j1nl2gdh0000gn/T/ipykernel_33511/2587967697.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xy[['BNN', 'DE', 'GP', 'RF']] = pd.get_dummies(selection['surrogate'])[['BNN', 'DE', 'GP', 'RF']]
/var/folders/pg/8kgknkg97zgbj393j1nl2gdh0000gn/T/ipykernel_3

In [30]:
import statsmodels.api as sm
aggregated_synth = aggregate_synthetic_results(synth_results)
selection = aggregated_synth[aggregated_synth['acquisition'] != "RS"]
xy = selection[['calibration_mse', 'tot_regret_pool']]
xy[['BNN', 'DE', 'GP', 'RF']] = pd.get_dummies(selection['surrogate'])[['BNN', 'DE', 'GP', 'RF']]
for model in set(aggregated_synth['surrogate']):
    xy[model] = xy[model]*xy['calibration_mse']
xy[['BNN_intercept', 'DE_intercept', 'GP_intercept', 'RF_intercept']] = pd.get_dummies(selection['surrogate'])[['BNN', 'DE', 'GP', 'RF']]
model = sm.OLS(xy['tot_regret_pool'], xy[['BNN', 'DE', 'GP', 'RF', 'BNN_intercept', 'DE_intercept', 'GP_intercept', 'RF_intercept']])
results=model.fit()
print(results.summary())
#scipy.stats.linregress(aggregated_synth['calibration_mse'], aggregated_synth['inst_regret_pool'])

                            OLS Regression Results                            
Dep. Variable:        tot_regret_pool   R-squared:                       0.119
Model:                            OLS   Adj. R-squared:                  0.119
Method:                 Least Squares   F-statistic:                     593.5
Date:                Tue, 14 Feb 2023   Prob (F-statistic):               0.00
Time:                        17:51:12   Log-Likelihood:            -1.7418e+05
No. Observations:               30720   AIC:                         3.484e+05
Df Residuals:                   30712   BIC:                         3.484e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
BNN            -459.0202     17.384    -26.404

/var/folders/pg/8kgknkg97zgbj393j1nl2gdh0000gn/T/ipykernel_33511/841702131.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xy[['BNN', 'DE', 'GP', 'RF']] = pd.get_dummies(selection['surrogate'])[['BNN', 'DE', 'GP', 'RF']]
/var/folders/pg/8kgknkg97zgbj393j1nl2gdh0000gn/T/ipykernel_33511/841702131.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xy[['BNN', 'DE', 'GP', 'RF']] = pd.get_dummies(selection['surrogate'])[['BNN', 'DE', 'GP', 'RF']]
/var/folders/pg/8kgknkg97zgbj393j1nl2gdh0000gn/T/ipykernel_335